In [1]:
import pandas as pd 
import numpy as np
import nltk
import re
from textblob import TextBlob

## Data Cleaning and Preprocessing

In [4]:
## Extrapolate random sample from walmart dataset
walmart_df = pd.read_csv('aldi.csv')


np.random.seed(2024)
idx_walmart = np.random.randint(low=0, high=len(walmart_df), size = 10)
samp_walmart_df = walmart_df.iloc[idx_walmart]

samp_walmart_df

,company,title,main_comment,invited_or_not,rating
2656,Aldi UK,Great service,Goods ordered online are not only free deliver...,0,5
2688,Aldi UK,We have received excellent service from …,We have received excellent service from the Ma...,0,5
2587,Aldi UK,great customer service as i was not…,great customer service as i was not happy with...,0,5
1572,Aldi UK,Don't get sucked into the hype,"Out of all the supermarkets, Aldi is my least ...",0,1
2494,Aldi UK,Goodbye Aldi,I am beginning to believe that Aldi train thei...,0,1
183,Aldi UK,Fuming is not the word!!!,"Purchashed the egg chair this morning, only fo...",0,1
3199,Aldi UK,Is this the Worlds worst supermarket?,"The food is Aldi is of very low quality, altho...",0,1
316,Aldi UK,Absolutely terrible service received my…,Absolutely terrible service received my ordere...,0,1
2275,Aldi UK,AVOID FRESH PRODUCE LIKE THE PLAGUE!!!!,AVOID FRESH PRODUCE LIKE THE PLAGUE!!!!my frie...,0,1
74,Aldi UK,Aberdeen Angus Burgers ....,Aberdeen Angus Burgers ..... x4 4oz absolutely...,0,1


#### Removing NaN

In [5]:
all_samples_df = samp_walmart_df.dropna().reset_index()
all_samples_df.isna().sum()

index             0
company           0
title             0
main_comment      0
invited_or_not    0
rating            0
dtype: int64

#### Removing unwanted characters

In [6]:
# initializing bad_chars_list

for row in range(len(all_samples_df)):
    all_samples_df['main_comment'][row] = re.sub('[^A-Za-z0-9 ,.!?]+', '', all_samples_df['main_comment'][row])
    all_samples_df['main_comment'][row] = all_samples_df['main_comment'][row].replace('...','.')
    all_samples_df['main_comment'][row] = all_samples_df['main_comment'][row].replace('..','.')
    all_samples_df['main_comment'][row] = all_samples_df['main_comment'][row].replace('!!!','!')
    all_samples_df['main_comment'][row] = all_samples_df['main_comment'][row].replace('!!','!')
    all_samples_df['main_comment'][row] = all_samples_df['main_comment'][row].replace('.','. ') 
all_samples_df['main_comment']

C:\Users\Marselo\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Marselo\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Marselo\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Marselo\anaconda3\lib\site-packages\ipy

0    Goods ordered online are not only free deliver...
1    We have received excellent service from the Ma...
2    great customer service as i was not happy with...
3    Out of all the supermarkets, Aldi is my least ...
4    I am beginning to believe that Aldi train thei...
5    Purchashed the egg chair this morning, only fo...
6    The food is Aldi is of very low quality, altho...
7    Absolutely terrible service received my ordere...
8    AVOID FRESH PRODUCE LIKE THE PLAGUE!my friend ...
9    Aberdeen Angus Burgers . .  x4 4oz absolutely ...
Name: main_comment, dtype: object

#### Fixing Incorrect Spelling

#### Tokenizing Sentence

In [7]:
from nltk.tokenize import sent_tokenize
row_tokenized = []
for row in range(len(all_samples_df)):
    row_tokenized += sent_tokenize(all_samples_df['main_comment'][row])

#remove the list with single sentence (we need opinion pairs)
clean_row_tokenized = []
for sent in row_tokenized:
    if len(re.findall(r'\w+', sent)) > 1:
        clean_row_tokenized.append(sent)
len(clean_row_tokenized)
#pd.DataFrame(clean_row_tokenized).to_csv('clean_sentence.csv', index = True)

65

In [8]:
clean_row_tokenized[1]

'We have received excellent service from the Management and team at our local Harworth South Yorkshire store, particularly this week when Jane, the Manager, took the trouble to find a product which had been sold out and was not available on line.'

## Building Universal Dependency Path

In [9]:
!python -m spacy download en_core_web_lg
#!python -m spacy download en_core_web_sm
import nltk
#nltk.download('vader_lexicon')

import spacy
nlp = spacy.load("en_core_web_lg")

from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


2021-06-08 10:16:31.667190: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-06-08 10:16:31.668061: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [10]:
def find_opinion_pairs_3(doc):
    # find roots of all entities in the text
    cleaned_rule3_pairs = []
    rule3_pairs = []
    #print(doc)
    add_no_det = False
    add_neg_pfx = False
    for token in doc:
        if token.pos_ == "VERB":
            
            children = token.children
            M = "999"
            A = "999"
            add_neg_pfx = False
            for child in children:
                
                #Finding the modifier
                if child.dep_ == "ccomp" and child.pos_ != 'AUX' and  child.pos_ != 'ADV':
                    M = child.text
                if child.dep_ == "acomp":
                    M = child.text

     
                #Finding the aspect
                if (child.dep_ == "nsubj" and child.pos_ == "PROPN") or (child.dep_ == "nsubj" and child.pos_ == "NOUN"):
                    for w in child.ancestors:
                        for w in w.children:
                            if w.dep_ == "xcomp":
                                A = child.text
                                for w in w.children:
                                    if w.dep_ =="acomp":
                                        M = w.text

                            if w.dep_ != "xcomp" :
                                to_be = ['is', 'am', 'are', 'were', 'was']
                                if token.text not in to_be:
                                    A = child.text
                                    for w in w.children:
                                        if w.dep_ == "compound":
                                            comp = w.text
                                            A = comp + " " + A
                                        M = token.text
                if child.dep_ == 'pobj' and child.pos_ == 'NOUN':
                    M = token.text
                    for w in child.ancestors:
                        for w in w.children:
                            if w.dep_ == 'advmod':
                                M = w.text + " " + M
                    A = child.text
                    for w in child.children:
                        if w.dep_ == "compound":
                            comp = w.text
                            A = comp + " " + A


                if child.dep_ == "nsubjpass" and child.pos_ == 'NOUN':
                    A = child.text
                    M = token.text
                    for w in child.children:
                        if w.dep_ == "compound":
                            comp = w.text
                            A = comp + " " + A   

                if child.dep_ =="dobj" and child.pos_ != 'PRON' and child.pos_ != 'NUM' and child.pos_ != 'DET':
                    A = child.text
                    M = token.text
                    for w in child.children:
                        if w.dep_ == "compound":
                            comp = w.text
                            A = comp + " " + A                    
                        if w.dep_ == "amod":
                            M = w.text
                        for w in w.children:
                            if w.dep_ == "advmod":
                                M = w.text + " " + M
                                
                if child.dep_ == 'prep':
                    M = token.text
                    for w in child.children:
                        if w.dep_ == 'pobj' and w.pos_ =='NOUN':
                            A = w.text
                            for w in w.children:
                                if w.dep_ == "compound":
                                    comp = w.text
                                    A = comp + " " + A    
          
                #Identify negation
                if child.dep_ == "neg":
                    neg_pfx = "not"
                    add_neg_pfx = True
                if child.dep_ =="aux" and child.pos_ == 'MD':
                    for w in child.ancestors:
                        for w in w.children:
                            if w.text in ('have'):
                                add_neg_pfx = True
                            if w.dep_ == 'neg':
                                add_neg_pfx = False
                            else:
                                add_neg_pfx = True
                if child.dep_ =="aux" and child.pos_ == 'VERB':
                    add_neg_pfx = True   
                    
            if add_neg_pfx == True and M != "999":
                M = "not" + " " + M

            if M != "999" and A != "999" and sid.polarity_scores(M)['compound'] != 0:
                rule3_pairs.append((A, M, sid.polarity_scores(M)['compound']))
 

        if token.pos_ == "AUX":
            children = token.children
            M = "999"
            A = "999"
            add_neg_pfx = False
            for child in children:
                #Finding the modifier
                if child.dep_ == "acomp" and child.pos_ == "VERB":
                    M = child.text
                    for w in child.children:
                        if w.dep_ == "advmod":
                            M = w.text + " " +  M

                if child.dep_ == "acomp" and child.pos_ != "VERB":
                    M = child.text
                    for w in child.children:
                        if w.dep_ == "advmod":
                            M = w.text + " " +  M  
                    for w in child.children: 
                        if w.dep_ == "prep":
                            for w in w.children:
                                if w.dep_ == "pobj":
                                    A = w.text
                                    for w in w.children:
                                        if w.dep_ == "compound":
                                            comp = w.text
                                            A = comp + " " + A
                #Finding the aspect
                if (child.dep_ == "nsubj" and child.pos_ == "NOUN") or (child.dep_ == "nsubj" and child.pos_ == "PROPN"):
                    A = child.text
                    for w in child.children:
                        if w.dep_ == "compound":
                            comp = w.text
                            A = comp + " " + A

                #Identify negation

                if child.dep_ == "neg":
                    neg_pfx = "not"
                    add_neg_pfx = True
 
   
            if add_neg_pfx == True and M != "999":
                M = "not" + " " + M
  
            if M != "999" and A != "999" and sid.polarity_scores(M)['compound'] != 0:
                rule3_pairs.append((A, M, sid.polarity_scores(M)['compound']))

        if token.pos_ == "NOUN":
            children = token.children
            M = "999"
            A = "999"
            add_neg_pfx = False
            for child in children:
                #Finding the modifier
                if child.dep_ == "amod":
                    M = child.text
                    for w in child.children:
                        if w.dep_ == "advmod":
                            M = w.text + " " + child.text
                      
                #Finding the aspect
                    A = token.text
                    
                    for w in child.ancestors:
                        if w.pos_ == 'VERB':
                            for w in w.children:
                                if w.dep_ == 'neg' and w.pos_ == 'PART':
                                    add_neg_pfx = True

                
                if child.dep_ == "compound" and A != "999":
                    comp = child.text
                    A = comp + " " + A
                    
                if child.dep_ == "det" and child.text == "no":
                    add_no_det = True
                    no_det = child.text
                    M = no_det + " " + token.text
           
                if child.dep_ == "prep" and add_no_det == True:
                    for w in child.children:
                        if w.dep_ == "pobj" and w.pos_ == 'NOUN':
                            A = w.text
                            if w.text == "am" or w.text == "pm": #time
                                A = 'time'
                            for w in w.children:
                                if w.dep_ == "compound":
                                    comp = w.text
                                    A = comp + " " + A
                if child.dep_ =='relcl':
                    A = token.text
                    for w in child.children:
                        ls_of_dep = [w.dep_ for w in child.children]
                        if "nsubj" in ls_of_dep:
                            M = child.text
 
                        if "aux" in ls_of_dep:
                            if w.dep_ == 'dobj':
                                A = w.text



            #Identify negation

            if add_neg_pfx == True and M != "999":
                M = "not" + " " + M

            if M != "999" and A != "999" and sid.polarity_scores(M)['compound'] != 0:
                rule3_pairs.append((A, M, sid.polarity_scores(M)['compound']))
                    
        if token.pos_ == "ADJ":
            children = token.children
            M = token.text
            A = "999"
            add_neg_pfx = False
            for child in children:
                #Finding the modifier
                if child.dep_ == "advmod":
                    M = child.text + " " + M

                #finding the aspect
                if child.dep_ == "prep":
                    for w in child.children:
                        if w.dep_ == "pobj" and w.pos_ in ('NOUN', 'PROPN'):
                            A = w.text
                            for w in w.children:
                                if w.dep_ =="compound":
                                    comp = w.text
                                    A = comp + " " + A
                        

                for w in child.ancestors:
                    if w.pos_ == 'AUX':
                        for w in w.children:
                            if w.dep_ =='neg':
                                add_neg_pfx = True

                    if w.pos_ == 'VERB':
                        for w in w.children:
                            if w.dep_ =='neg':
                                add_neg_pfx = True
                #Identify negation
                if child.dep_ == "neg":
                    neg_pfx = "not"
                    add_neg_pfx = True
   
            if add_neg_pfx == True and M != "999":
                M = "not" + " " + M

            if M != "999" and A != "999" and sid.polarity_scores(M)['compound'] != 0:
                rule3_pairs.append((A, M, sid.polarity_scores(M)['compound']))
                
    #Removing duplicates            
    for ls in rule3_pairs:
        if ls not in cleaned_rule3_pairs:
            cleaned_rule3_pairs.append(ls)
    return cleaned_rule3_pairs

print(find_opinion_pairs_3(nlp("They do not have a bad shipping")))
print(find_opinion_pairs_3(nlp("")))
#print(find_opinion_pairs_3(nlp("No one should feel victimised in a supermarket of all places, being ignored like hes invisible isnt acceptable")))
#print(find_opinion_pairs_3(nlp("I have never come accross such a good service")))

# I have never come accross such a good service
#remove I, me, it, anything, noone, anybody, this, that, us, you

[('shipping', 'not bad', 0.431)]
[]


### Proven working find_opinion_pairs_3

## Trial Run with the real dataset

In [11]:
opinion_pairs = []
for sent in clean_row_tokenized:
    sent = ' '.join(re.findall(r"[a-zA-Z,!']+", sent))
    opinion_pair = find_opinion_pairs_3(nlp(sent))
    opinion_pairs.append(opinion_pair)
    
#opinion_pairs[1]    
result_df = pd.DataFrame(clean_row_tokenized)
result_df['opinion_pairs'] = opinion_pairs
#result_df.to_csv('second_run.csv', index = False)

##### Generating dataset for evaluation

In [12]:
clean_evaluation_index = []
for a in range(len(result_df)):
    for b in range(len(result_df['opinion_pairs'][a])):
        if result_df['opinion_pairs'][a]:
            if result_df['opinion_pairs'][a][b][2] != 0.0:
                if len(result_df['opinion_pairs'][a]) >= 1:
                    clean_evaluation_index.append(a)

#remove the duplicate numbers
clean_evaluation_index = list(set(clean_evaluation_index))

#save it into csv file
evaluation_df = result_df.iloc[clean_evaluation_index]
#evaluation_df.to_csv('aspect_term_extraction_evaluation.csv', index = False)
print(len(clean_evaluation_index))

25


In [13]:
sent_opinion_pairs = []
for a in range(len(result_df)):
    opin_pairs = []
    for b in range(len(result_df['opinion_pairs'][a])):
        if result_df['opinion_pairs'][a]:
            if len(result_df['opinion_pairs'][a]) >= 1:
                if result_df['opinion_pairs'][a][b][2] != 0.0:
                    opin_pair = result_df['opinion_pairs'][a][b]
                    opin_pairs.append(opin_pair)
                sent_opinion_pairs.append([result_df[0][a], opin_pairs])
#remove the duplicate numbers
#save it into csv file
evaluation_df = pd.DataFrame(sent_opinion_pairs).drop_duplicates(subset = 0, keep = 'first').reset_index()
evaluation_df
#evaluation_df.to_csv('aspect_term_extraction_evaluation_3.csv', index = False)
evaluation_df

,index,0,1
0,0,Goods ordered online are not only free deliver...,"[(delivery, free, 0.5106)]"
1,1,We have received excellent service from the Ma...,"[(service, excellent, 0.5719)]"
2,2,I think that is excellent service and maintain...,"[(service, excellent, 0.5719)]"
3,3,"Thank you Jane and your team, keep up the good...","[(work!Kay, good, 0.4404)]"
4,4,great customer service as i was not happy with...,"[(customer service, great, 0.6249), (item, not..."
5,7,They are such scammers with their highly misle...,"[(advertising, highly misleading, -0.4576)]"
6,8,Theres no alternative in store though.,"[(store, no alternative, -0.296)]"
7,9,The full special buy ranges can only be seen i...,"[(buy ranges, special, 0.4019), (journey, wast..."
8,11,I asked her to slow down a little as products ...,"[(products, crushed, -0.4215)]"
9,12,I explained that I could pack the bags all set...,"[(trolley, ready, 0.3612)]"


## Aggregation

#### Generating clean list

In [14]:
opinion_pairs_ls = evaluation_df[1].tolist()
clean_opinion_pairs_ls = []
for i in range(len(opinion_pairs_ls)):
    if len(opinion_pairs_ls[i]) != 0:
        for j in range(len(opinion_pairs_ls[i])):
            if opinion_pairs_ls[i][j][2] != 0.0:                
                clean_opinion_pairs_ls.append(opinion_pairs_ls[i][j])
                
clean_opinion_pairs_df = pd.DataFrame(clean_opinion_pairs_ls)

#clean_opinion_pairs_df.to_csv('clean_list_4.csv', index = True)

In [15]:
clean_opinion_pairs_df

,0,1,2
0,delivery,free,0.5106
1,service,excellent,0.5719
2,service,excellent,0.5719
3,work!Kay,good,0.4404
4,customer service,great,0.6249
5,item,not happy,-0.4585
6,item,not faulty,0.2411
7,advertising,highly misleading,-0.4576
8,store,no alternative,-0.2960
9,buy ranges,special,0.4019


In [16]:
cleaned_opinion_pair = []
for row in range(len(clean_opinion_pairs_ls)):
    cleaned_aspect =  ' '.join(re.findall(r"[a-zA-Z]+", clean_opinion_pairs_ls[row][0]))
    cleaned_aspect = cleaned_aspect.lower()
    
    
    opinion =  ' '.join(re.findall(r"[a-zA-Z]+", clean_opinion_pairs_ls[row][1]))
    opinion = opinion.lower()
    cleaned_opinion_pair.append((cleaned_aspect, opinion, clean_opinion_pairs_ls[row][2]))
    
cleaned_opinion_pair

[('delivery', 'free', 0.5106),
 ('service', 'excellent', 0.5719),
 ('service', 'excellent', 0.5719),
 ('work kay', 'good', 0.4404),
 ('customer service', 'great', 0.6249),
 ('item', 'not happy', -0.4585),
 ('item', 'not faulty', 0.2411),
 ('advertising', 'highly misleading', -0.4576),
 ('store', 'no alternative', -0.296),
 ('buy ranges', 'special', 0.4019),
 ('journey', 'wasted', -0.4939),
 ('products', 'crushed', -0.4215),
 ('trolley', 'ready', 0.3612),
 ('bags', 'drop', -0.2732),
 ('easter', 'no one', -0.296),
 ('quality', 'very low', -0.3384),
 ('tomatoes', 'nice', 0.4215),
 ('makers', 'fighting', -0.3612),
 ('reason', 'dislike', -0.3818),
 ('aldi', 'dislike', -0.3818),
 ('result', 'rude', -0.4588),
 ('quality', 'satisfactory', 0.3612),
 ('place', 'pleasant', 0.5106),
 ('one', 'not nice', -0.3252),
 ('aldi', 'won', 0.5719),
 ('shops', 'clean', 0.4019),
 ('staff', 'so friendly', 0.5413),
 ('shopping experience', 'nice', 0.4215),
 ('service', 'absolutely terrible', -0.5256),
 ('servic

In [17]:
clean_opinion_pairs_ls[0][0]

'delivery'

### Categorizing the extracted terms

In [18]:
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic

# Wordnet information content file
brown_ic = wordnet_ic.ic('ic-brown.dat')

dog = wn.synsets('dog')[5]
planet = wn.synsets('planet')[0]
print(wn.jcn_similarity(dog, planet, ic=brown_ic))

0.07177678389304161


In [19]:
from collections import Iterable

def flatten(lis):
     for item in lis:
        if isinstance(item, Iterable) and not isinstance(item, str):
            for x in flatten(item):
                yield x
        else:        
            yield item

def synset_generator(opinion_pairs):
    ls_synsets = []
    print(opinion_pairs)
    for word in opinion_pairs.split():
        ls_synsets.append(wn.synsets(word))
        ls_synsets = list(flatten(ls_synsets))
         
    return ls_synsets


C:\Users\Marselo\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  """Entry point for launching an IPython kernel.


In [20]:
list(flatten(['blue', ['ball']]))

['blue', 'ball']

In [21]:
pair1 = "credit card"
pair2 = "customer service"
list(synset_generator(pair1))

credit card


[Synset('recognition.n.03'),
 Synset('credit.n.02'),
 Synset('credit.n.03'),
 Synset('credit.n.04'),
 Synset('credit.n.05'),
 Synset('credit.n.06'),
 Synset('citation.n.03'),
 Synset('credit.n.08'),
 Synset('credit_rating.n.01'),
 Synset('credit.v.01'),
 Synset('accredit.v.03'),
 Synset('credit.v.03'),
 Synset('credit.v.04'),
 Synset('card.n.01'),
 Synset('card.n.02'),
 Synset('card.n.03'),
 Synset('card.n.04'),
 Synset('wag.n.01'),
 Synset('poster.n.01'),
 Synset('calling_card.n.02'),
 Synset('card.n.08'),
 Synset('menu.n.01'),
 Synset('batting_order.n.01'),
 Synset('circuit_board.n.01'),
 Synset('tease.v.07'),
 Synset('card.v.02')]

In [22]:
def jcn_score(pair1, pair2):

    #print("pair opinion 1: %s\n" % pair1)
    #print("pair opinion 2: %s\n" % pair2)

    # For each synset in the first sentence...
    if len(list(synset_generator(pair1))) != 0 and  len(list(synset_generator(pair2))) != 0:

        jcn_scores = []
        syn1 = []
        syn2 = []
        for synset1 in list(synset_generator(pair1)):

            max_score = 0
            synsetScore = 0

            # For each synset in the second sentence...
            for synset2 in list(synset_generator(pair2)):


                # Only compare synsets with the same POS tag. Word to word knowledge
                # measures cannot be applied across different POS tags.
                if (synset1.pos() == synset2.pos()) and (synset1.pos() == 'n' and synset2.pos() == 'n'):


                    # Note below is the call to path_similarity mentioned above. 
                    synsetScore = wn.jcn_similarity(synset1, synset2, ic = brown_ic, verbose = False)

                    if synsetScore != None:
                        #print("Path Score %0.2f: %s vs. %s" % (synsetScore, synset1, synset2))
                        jcn_scores.append(synsetScore)
                        syn1.append(synset1)
                        syn2.append(synset2)

                    # If there are no similarity results but the SAME WORD is being
                    # compared then it gives a max score of 1.
                    elif synset1.name().split(".")[0] == synset2.name().split(".")[0]:
                        synsetScore = 1
                        #print("Path MAX-Score %0.2f: %s vs. %s" % (synsetScore, synset1, synset2))
                        jcn_scores.append(synsetScore)
                        syn1.append(synset1)
                        syn2.append(synset2)


            #generate the max value

        if(len(jcn_scores) > 0):
            max_score = max(jcn_scores)


        #create temporary list
        temp_list = list (zip(syn1, syn2, jcn_scores ))

        highest_synset = []
        for a in range(len(temp_list)):
            for b in range(len(temp_list[a])):
                if temp_list[a][2] == max_score:
                    highest_synset = temp_list[a]
                    return highest_synset

In [23]:
from nltk.corpus import wordnet as wn
jcn_score('monkey', 'ape')

monkey
ape
monkey
ape
ape
ape
ape


(Synset('monkey.n.01'), Synset('ape.n.01'), 0.08429275499544277)

### Lemmatization

In [24]:
clean_opinion_pairs_ls[1][0]

'service'

In [25]:
from nltk.stem.wordnet import WordNetLemmatizer
import re
lemmatizer  = WordNetLemmatizer()
clean_lemmatized_ls = []
for ls in range(len(cleaned_opinion_pair)):
    for words in [cleaned_opinion_pair[ls][0]]:
        clean_word = re.findall(r"[a-zA-Z]+", words)
        lemmatized = [lemmatizer.lemmatize(word) for word in clean_word]
        lemmatized = ' '.join(lemmatized)
        lemmatized = lemmatized.lower()
        clean_lemmatized_ls.append(lemmatized)

clean_lemmatized_ls = list(zip(clean_opinion_pairs_ls, clean_lemmatized_ls))
clean_lemmatized_ls[1][1]

'service'

In [26]:
clean_lemmatized_ls

[(('delivery', 'free', 0.5106), 'delivery'),
 (('service', 'excellent', 0.5719), 'service'),
 (('service', 'excellent', 0.5719), 'service'),
 (('work!Kay', 'good', 0.4404), 'work kay'),
 (('customer service', 'great', 0.6249), 'customer service'),
 (('item', 'not happy', -0.4585), 'item'),
 (('item', 'not faulty', 0.2411), 'item'),
 (('advertising', 'highly misleading', -0.4576), 'advertising'),
 (('store', 'no alternative', -0.296), 'store'),
 (('buy ranges', 'special', 0.4019), 'buy range'),
 (('journey', 'wasted', -0.4939), 'journey'),
 (('products', 'crushed', -0.4215), 'product'),
 (('trolley', 'ready', 0.3612), 'trolley'),
 (('bags', 'drop', -0.2732), 'bag'),
 (('easter', 'no one', -0.296), 'easter'),
 (('quality', 'very low', -0.3384), 'quality'),
 (('tomatoes', 'nice', 0.4215), 'tomato'),
 (('makers', 'fighting', -0.3612), 'maker'),
 (('reason', 'dislike', -0.3818), 'reason'),
 (('Aldi', 'dislike', -0.3818), 'aldi'),
 (('result', 'rude', -0.4588), 'result'),
 (('quality', 'sati

### Aspect Clustering

### Matrix creation

In [27]:
all_aspects = []
all_synsets = []

for ls1 in range(len(clean_lemmatized_ls)):
    one_aspect = []
    aspect1 = clean_lemmatized_ls[ls1][1]
    for ls2 in range(len(clean_lemmatized_ls)):
        aspect2 = clean_lemmatized_ls[ls2][1]
        if jcn_score(aspect1, aspect2) is None:
            one_aspect.append(0)
        if jcn_score(aspect1, aspect2):
            syn1, syn2, score = jcn_score(aspect1, aspect2)

            if score >= 1:
                one_aspect.append(1)
            else:
                one_aspect.append(score)
    all_synsets.append(syn1)
    all_aspects.append(one_aspect)
            

delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
service
delivery
service
service
service
service
service
service
service
delivery
service
delivery
service
service
service
service
service
service
service
delivery
service
delivery
service
service
service
service
service
service
service
delivery
service
delivery
service
service
service
service
service
service
service
delivery
service
delivery
service
service
service
service
service
service
service
delivery
service
delivery
service
service
service
service
service
service
service
delivery
work kay
delivery
work kay
work kay
work kay
work kay
work kay
work kay
work kay
delivery
work kay
delivery
work kay
work kay
work kay
work kay
work kay
work kay
work kay
delivery
work kay
delivery
work kay
work kay
work kay
work

buy range
service
buy range
service
buy range
buy range
buy range
buy range
buy range
buy range
buy range
buy range
buy range
buy range
buy range
buy range
buy range
buy range
buy range
buy range
buy range
buy range
service
journey
service
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
service
journey
service
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
service
journey
service
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
journey
service
product
service
product
product
product
product
product
product
product
product
product
product
product
product
product
product
product
product
product
product
service
product
service
product
product
product
product
product
product
product
product
product
product
product


sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
service
sickness
service
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
service
sickness
service
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
sickness
service
mistake
service
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
service
mistake
service
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
service
mistake
service
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
mistake
m

trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
service
trolley
service
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
service
trolley
service
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
trolley
service
bag
service
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
service
bag
service
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
service
bag
service
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
bag
service
easter
service
easter
easter
easter
easter
easter
easter
easter
easter
easter
easter
easter
easter
easter
easter
easter
easter
easter
easter
service
easter
service
easter
easter
easter
easter
easter
easter
easter
easter
easter
easter
easter
easter
easter
easter
easter
easter


product
product
product
product
product
product
product
product
product
product
service
product
service
product
product
product
product
product
product
product
product
product
product
product
product
product
product
product
product
product
product
service
account
service
account
account
account
account
account
account
account
account
account
account
account
account
account
account
account
account
account
account
service
account
service
account
account
account
account
account
account
account
account
account
account
account
account
account
account
account
account
account
account
service
account
service
account
account
account
account
account
account
account
account
account
account
account
account
account
account
account
account
account
account
work kay
delivery
work kay
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
deliv

...

item
item
item
item
item
produce
item
produce
item
item
item
item
item
item
item
item
produce
item
produce
item
item
item
item
item
item
item
item
produce
advertising
produce
advertising
advertising
advertising
advertising
advertising
advertising
advertising
advertising
produce
advertising
produce
advertising
advertising
advertising
advertising
advertising
advertising
advertising
advertising
produce
advertising
produce
advertising
advertising
advertising
advertising
advertising
advertising
advertising
advertising
produce
store
produce
store
store
store
store
store
store
store
store
produce
store
produce
store
store
store
store
store
store
store
store
produce
store
produce
store
store
store
store
store
store
store
store
produce
buy range
produce
buy range
buy range
buy range
buy range
buy range
buy range
buy range
buy range
produce
buy range
produce
buy range
buy range
buy range
buy range
buy range
buy range
buy range
buy range
produce
buy range
produce
buy range
buy range
buy range
buy

place
sickness
place
sickness
place
place
place
sickness
one
sickness
one
one
one
sickness
one
sickness
one
one
one
sickness
one
sickness
one
one
one
sickness
aldi
sickness
aldi
sickness
shop
sickness
shop
shop
shop
sickness
shop
sickness
shop
shop
shop
sickness
shop
sickness
shop
shop
shop
sickness
staff
sickness
staff
staff
staff
sickness
staff
sickness
staff
staff
staff
sickness
staff
sickness
staff
staff
staff
sickness
shopping experience
sickness
shopping experience
shopping experience
shopping experience
sickness
shopping experience
sickness
shopping experience
shopping experience
shopping experience
sickness
shopping experience
sickness
shopping experience
shopping experience
shopping experience
sickness
service
sickness
service
service
service
sickness
service
sickness
service
service
service
sickness
service
sickness
service
service
service
sickness
service
sickness
service
service
service
sickness
service
sickness
service
service
service
sickness
service
sickness
service
serv

bag
blame
bag
bag
bag
bag
bag
bag
blame
bag
blame
bag
bag
bag
bag
bag
bag
blame
easter
blame
easter
easter
easter
easter
easter
easter
blame
easter
blame
easter
easter
easter
easter
easter
easter
blame
easter
blame
easter
easter
easter
easter
easter
easter
blame
quality
blame
quality
quality
quality
quality
quality
quality
blame
quality
blame
quality
quality
quality
quality
quality
quality
blame
quality
blame
quality
quality
quality
quality
quality
quality
blame
tomato
blame
tomato
tomato
tomato
tomato
tomato
tomato
blame
tomato
blame
tomato
tomato
tomato
tomato
tomato
tomato
blame
tomato
blame
tomato
tomato
tomato
tomato
tomato
tomato
blame
maker
blame
maker
maker
maker
maker
maker
maker
blame
maker
blame
maker
maker
maker
maker
maker
maker
blame
maker
blame
maker
maker
maker
maker
maker
maker
blame
reason
blame
reason
reason
reason
reason
reason
reason
blame
reason
blame
reason
reason
reason
reason
reason
reason
blame
reason
blame
reason
reason
reason
reason
reason
reason
blame
aldi


blame
blame
blame
blame
product
product
product
product
product
product
product
product
product
product
product
product
product
product
product
product
product
product
product
product
product
product
product
product
product
product
product
product
account
product
account
account
account
account
account
account
product
account
product
account
account
account
account
account
account
product
account
product
account
account
account
account
account
account
account
delivery
account
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
account
delivery
account
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
account
delivery
account
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
delivery
account
service
account
service
service
service
service
service
service
service
service
ser

staff
staff
staff
staff
staff
staff
staff
account
staff
account
staff
staff
staff
staff
staff
staff
staff
staff
staff
staff
staff
staff
staff
staff
account
staff
account
staff
staff
staff
staff
staff
staff
staff
staff
staff
staff
staff
staff
staff
staff
account
shopping experience
account
shopping experience
shopping experience
shopping experience
shopping experience
shopping experience
shopping experience
shopping experience
shopping experience
shopping experience
shopping experience
shopping experience
shopping experience
shopping experience
shopping experience
account
shopping experience
account
shopping experience
shopping experience
shopping experience
shopping experience
shopping experience
shopping experience
shopping experience
shopping experience
shopping experience
shopping experience
shopping experience
shopping experience
shopping experience
shopping experience
account
shopping experience
account
shopping experience
shopping experience
shopping experience
shopping experienc

In [28]:
all_aspects

[[1,
  1,
  1,
  0.12386893236031576,
  1,
  0.08348565920720627,
  0.08348565920720627,
  0.10655074179712883,
  0.0696981396468037,
  0.07618974166875081,
  0.1511521400647245,
  0.07573639420835165,
  0.051893688730500015,
  0.07475949877840171,
  0.05394546333220039,
  0.09504746070306973,
  0.048964773097731444,
  0.08317576016501506,
  0.09430665514681408,
  0,
  0.09653206449353527,
  0.09504746070306973,
  0.09951168127545233,
  0.09030225314694831,
  0,
  0.07383735296441821,
  0.08182149587982587,
  0.0985201373771096,
  1,
  1,
  0.06558392884464373,
  0.07235769425674389,
  0.08964530304464824,
  0.074672569567331,
  0.07573639420835165,
  0.10730243096932272],
 [1,
  1,
  1,
  0.7768933718820784,
  1,
  0.08349164747091488,
  0.08349164747091488,
  0.07434239289793809,
  0.08309706354963352,
  0.09056762664334754,
  0.23315003299333276,
  0.10944644984081524,
  0.06577492190598956,
  0.08794954728912438,
  0.08164779150406241,
  0.2590737433891984,
  0.05995249757370266,
 

In [29]:
#aspect_array = np.load(r"C:\Users\Marselo\Documents\Github2\trustpilot2\online\results\aldi.npy")
aspect_array = np.array(all_aspects)
print(aspect_array.shape)

#normalize the array
from sklearn import preprocessing
norm_aspect_array = preprocessing.normalize(aspect_array)

#save the array
#np.save(r"C:\Users\Marselo\Documents\Github2\trustpilot2\online\results\costco.npy", norm_aspect_array)


(36, 36)


In [30]:
norm_aspect_array[0]

array([0.40106088, 0.40106088, 0.40106088, 0.04967898, 0.40106088,
       0.03348283, 0.03348283, 0.04273333, 0.0279532 , 0.03055672,
       0.06062121, 0.0303749 , 0.02081253, 0.02998311, 0.02163541,
       0.03811982, 0.01963785, 0.03335854, 0.03782271, 0.        ,
       0.03871523, 0.03811982, 0.03991024, 0.0362167 , 0.        ,
       0.02961327, 0.0328154 , 0.03951257, 0.40106088, 0.40106088,
       0.02630315, 0.02901984, 0.03595322, 0.02994825, 0.0303749 ,
       0.04303481])

# Determine number of Clusters

## Create labels/Cluster number

#### Create label

In [86]:
from sklearn_extra.cluster import KMedoids

n_cluster = int(aspect_array.shape[0]*0.5)
kmedoids = KMedoids(n_clusters = n_cluster, random_state=0, method = 'pam').fit(norm_aspect_array)
label = list(kmedoids.labels_)
pd.DataFrame(label).to_csv('label.csv', index = False)

## Assign label to cluster name

#### Synset Generator

In [87]:
len(clean_lemmatized_ls)

36

In [88]:
all_synsets[0]

Synset('manner_of_speaking.n.01')

#### Create cluster_names

In [89]:
cluster_names = []
center = kmedoids.cluster_centers_
center_ls = center.tolist()

no_dup = []
for elem in center_ls:
    if elem not in no_dup:
        no_dup.append(elem)

#remove duplicate values from center
non_dup_center = list(set(tuple(sub) for sub in center))
for a in range(len(norm_aspect_array)):
    for b in range(len(non_dup_center)):
        temp_array = norm_aspect_array[a] == non_dup_center[b]
        if False not in temp_array:
            if sum(norm_aspect_array[a]) == 0:
                cluster_names.append(["unidentified", label[a], all_synsets[a]])               
            else:
                cluster_name = clean_lemmatized_ls[a][1]
                cluster_names.append([cluster_name, label[a], all_synsets[a]])

'''                
cluster_names = pd.DataFrame(cluster_names).drop_duplicates(subset = 0, keep = 'first')
'''
cluster_names = pd.DataFrame(cluster_names).drop_duplicates(subset = 1, keep = 'first')

#cluster_names.to_csv('cluster_names.csv', index = False)

#### Assign cluster name to opinion pairs list

In [93]:
aspect_term_label = []
for a in range(len(clean_lemmatized_ls)):
    aspect_term_label.append(clean_lemmatized_ls[a][1])
    
aspect_term_label = pd.DataFrame(list(zip(aspect_term_label, label)))
aspect_term_label = pd.merge(aspect_term_label, cluster_names, left_on = 1, right_on = 1, how = 'left')
aspect_term_label['0_y']
#aspect_term_label.to_csv('aspect_term_label.csv', index = False)

0                 service
1                 service
2                 service
3                 product
4                 service
5                    item
6                    item
7            unidentified
8                   store
9               buy range
10                service
11                product
12                trolley
13                    bag
14                 easter
15                quality
16           unidentified
17                  maker
18                 reason
19           unidentified
20                product
21                quality
22                   item
23                    one
24           unidentified
25                  store
26                service
27    shopping experience
28                service
29                service
30                produce
31               sickness
32                mistake
33                  blame
34                product
35                 reason
Name: 0_y, dtype: object

#### Rate product aspects

In [94]:
sentiment_with_cluster_df[2]

0     0.5106
1     0.5719
2     0.5719
3     0.4404
4     0.6249
5    -0.4585
6     0.2411
7    -0.4576
8    -0.2960
9     0.4019
10   -0.4939
11   -0.4215
12    0.3612
13   -0.2732
14   -0.2960
15   -0.3384
16    0.4215
17   -0.3612
18   -0.3818
19   -0.3818
20   -0.4588
21    0.3612
22    0.5106
23   -0.3252
24    0.5719
25    0.4019
26    0.5413
27    0.4215
28   -0.5256
29   -0.2755
30    0.3182
31    0.4847
32   -0.2924
33   -0.2924
34    0.3724
35   -0.4588
Name: 2, dtype: float64

In [92]:
sentiment_with_cluster_df = clean_opinion_pairs_df
sentiment_with_cluster_df[0] = pd.DataFrame(clean_lemmatized_ls)[1]
sentiment_with_cluster_df['cluster'] = aspect_term_label['0_y']
sentiment_with_cluster_df['pos_or_neg'] = [1 if word > 0 else 0 for word in sentiment_with_cluster_df[2]]
sentiment_with_cluster_df.head(50)

,0,1,2,cluster,pos_or_neg
0,delivery,free,0.5106,service,1
1,service,excellent,0.5719,service,1
2,service,excellent,0.5719,service,1
3,work kay,good,0.4404,product,1
4,customer service,great,0.6249,service,1
5,item,not happy,-0.4585,item,0
6,item,not faulty,0.2411,item,1
7,advertising,highly misleading,-0.4576,unidentified,0
8,store,no alternative,-0.2960,store,0
9,buy range,special,0.4019,buy range,1


#### Generating dataset for evaluation

In [ ]:
evaluation_final_df = pd.DataFrame()
evaluation_final_df['sentence'] = evaluation_df.reset_index()[0]
evaluation_final_df['term'] = sentiment_with_cluster_df[0]
evaluation_final_df['sentiment_word'] = sentiment_with_cluster_df[1]
evaluation_final_df['cluster'] = sentiment_with_cluster_df['cluster']
#evaluation_final_df.to_csv("evaluation_final_df_3.csv", index = False)

#### Create a list of words associated with the synsets

In [95]:
variable = {'aesthetics': ["appearance", "sound", "taste", "smell", "packaging", "design", "shapes", "styles", "size" ], 
            'personal_interaction': ["courtesy", "politeness", "courteousness", "helpfulness", "staff", "responsiveness", 
                                 "assurance", "empathy", "trust", "confidence", "safety",  "availability", "knowledge", "friendliness", "promptness", "professionalism"],
            'policy': ["return", 'refund', 'order' 'cancellation', 'repair', 'warranty', 'exchange', 'payment', 'operating hours', 'cards', 
                  'parking', 'delivery', 'shipping'],
            'physical_aspect' : ['layout', 'design', 'cleanliness', 'convenience', 'assortment', 'tills', 'cashiers', 'fitting rooms', 'restroom', 'shopping bags', 'catalogues', 'movement', 
                            'cart', 'facilities', 'check-out', 'website', "place", "location", 'trolley'],
            'durability': ['defect', 'malfunction', 'lifetime', 'expiration', 'endurance'],
            'performance': ['performance'],
            'service_reliability': ['promise', 'efficiency'],
            'price': ['price', 'affordability', 'value', 'worth'],
            'problem_solving': ['complaint', 'report', 'customer service', 'call center', 'support'],
            'meet_customer_needs': ['needs', 'requirement', 'wants', 'satisfaction', 'complacency']}

dimension_dict = {'high_quality': ['aesthetics', 'personal_interaction', 'policy', 'physical_aspect', 'durability', 'performance',
                             'service_reliability'],
            'good_value': ['price'],
            'stands_behind': ['problem_solving'],
            'meet_customer_needs': ['meet_customer_needs']}

#### Create function for generating the maxixum score pairwise jcn_score

In [96]:
def jcn_score_2 (synset1, pair):
    if len(list(synset_generator(pair))):
        synsetScore = 0
        jcn_scores = []
        for synset2 in list(synset_generator(pair)):
            if synset1.pos() == synset2.pos() and (synset1.pos() == 'n' and synset2.pos() == 'n'):
                synsetScore = wn.jcn_similarity(synset1, synset2, ic = brown_ic)

                if synsetScore != None:
                    jcn_scores.append(synsetScore)

                elif synset1.name().split(".")[0] == synset2.name().split(".")[0]:
                    synsetScore = 1
                    #print("Path MAX-Score %0.2f: %s vs. %s" % (synsetScore, synset1, synset2))
                    jcn_scores.append(synsetScore)
                    syn1.append(synset1)
                    syn2.append(synset2)
                synsetScore = 0

            #generate the max value

        if(len(jcn_scores) > 0):
            max_score = max(jcn_scores)
            return max_score

In [97]:
all_aspect_var = []
for synset1 in cluster_names[2]:
    one_aspect = []
    for v in variable:
        cons_list = variable[v]
        for pair in cons_list:
            jcn_score = jcn_score_2 (synset1, pair)
            one_aspect.append(jcn_score)
    all_aspect_var.append(one_aspect)

appearance
appearance
sound
sound
taste
taste
smell
smell
packaging
packaging
design
design
shapes
shapes
styles
styles
size
size
courtesy
courtesy
politeness
politeness
courteousness
helpfulness
helpfulness
staff
staff
responsiveness
responsiveness
assurance
assurance
empathy
empathy
trust
trust
confidence
confidence
safety
safety
availability
availability
knowledge
knowledge
friendliness
friendliness
promptness
promptness
professionalism
professionalism
return
return
refund
refund
ordercancellation
repair
repair
warranty
warranty
exchange
exchange
payment
payment
operating hours
operating hours
cards
cards
parking
parking
delivery
delivery
shipping
shipping
layout
layout
design
design
cleanliness
cleanliness
convenience
convenience
assortment
assortment
tills
tills
cashiers
cashiers
fitting rooms
fitting rooms
restroom
restroom
shopping bags
shopping bags
catalogues
catalogues
movement
movement
cart
cart
facilities
facilities
check-out
website
website
place
place
location
location
tr

satisfaction
complacency
complacency
appearance
appearance
sound
sound
taste
taste
smell
smell
packaging
packaging
design
design
shapes
shapes
styles
styles
size
size
courtesy
courtesy
politeness
politeness
courteousness
helpfulness
helpfulness
staff
staff
responsiveness
responsiveness
assurance
assurance
empathy
empathy
trust
trust
confidence
confidence
safety
safety
availability
availability
knowledge
knowledge
friendliness
friendliness
promptness
promptness
professionalism
professionalism
return
return
refund
refund
ordercancellation
repair
repair
warranty
warranty
exchange
exchange
payment
payment
operating hours
operating hours
cards
cards
parking
parking
delivery
delivery
shipping
shipping
layout
layout
design
design
cleanliness
cleanliness
convenience
convenience
assortment
assortment
tills
tills
cashiers
cashiers
fitting rooms
fitting rooms
restroom
restroom
shopping bags
shopping bags
catalogues
catalogues
movement
movement
cart
cart
facilities
facilities
check-out
website
web

availability
knowledge
knowledge
friendliness
friendliness
promptness
promptness
professionalism
professionalism
return
return
refund
refund
ordercancellation
repair
repair
warranty
warranty
exchange
exchange
payment
payment
operating hours
operating hours
cards
cards
parking
parking
delivery
delivery
shipping
shipping
layout
layout
design
design
cleanliness
cleanliness
convenience
convenience
assortment
assortment
tills
tills
cashiers
cashiers
fitting rooms
fitting rooms
restroom
restroom
shopping bags
shopping bags
catalogues
catalogues
movement
movement
cart
cart
facilities
facilities
check-out
website
website
place
place
location
location
trolley
trolley
defect
defect
malfunction
malfunction
lifetime
lifetime
expiration
expiration
endurance
endurance
performance
performance
promise
promise
efficiency
efficiency
price
price
affordability
value
value
worth
worth
complaint
complaint
report
report
customer service
customer service
call center
call center
support
support
needs
needs
req

#### Clustering the identified term to the variable in the conceptual model 

In [98]:
variable_array = np.array(all_aspect_var)
print(variable_array.shape)
#check for null values 
variable_array = np.where(variable_array== None, 0, variable_array)

(18, 78)


In [99]:
pd.DataFrame(variable_array).describe()

,0,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,77
count,18.000000,18.000000,18.000000,18.000000,18.000000,18.00000,18.000000,18.000000,18.000000,18.000000,...,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000
unique,17.000000,17.000000,17.000000,17.000000,17.000000,17.00000,17.000000,17.000000,17.000000,17.000000,...,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000
top,0.078647,0.084981,0.067883,0.069653,0.115775,0.07598,0.085198,0.115068,0.071845,0.093066,...,0.157379,0.120212,0.069235,0.112224,0.083006,0.075692,0.067666,0.067039,0.060551,0.050048
freq,2.000000,2.000000,2.000000,2.000000,2.000000,2.00000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000


In [100]:
jcn_cutoff = 0.3
all_clusters = []
for a in range(len(variable_array)):
    clusters = []
    for b in range(len(variable_array[a])):
        if variable_array[a][b] >= jcn_cutoff:
            clus = 1
        else:
            clus = 0
        clusters.append(clus)
    all_clusters.append(clusters)
all_clusters

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0

In [101]:
len(cluster_names)

18

In [102]:
col_names = []
for a in variable:
    for b in variable[a]:
        col_names.append(b)
variable_df = pd.DataFrame(data = all_clusters, index = cluster_names[0], columns = col_names)
#variable_df.to_csv('cluster.csv', index = True)

#### Clustering the variable into the conceptual model

In [103]:
variable_names = []
for a in range(len(variable_df)):
    if sum(variable_df.iloc[a][variable['aesthetics']]) > 0:
        variable_names.append('aesthetics')
    elif sum(variable_df.iloc[a][variable['personal_interaction']]) > 0:
        variable_names.append('personal_interaction')
    elif sum(variable_df.iloc[a][variable['policy']]) > 0:
        variable_names.append('policy')
    elif sum(variable_df.iloc[a][variable['physical_aspect']]) > 0:
        variable_names.append('physical_aspect')
    elif sum(variable_df.iloc[a][variable['durability']]) > 0:
        variable_names.append('durability')
    elif sum(variable_df.iloc[a][variable['performance']]) > 0:
        variable_names.append('performance')
    elif sum(variable_df.iloc[a][variable['service_reliability']]) > 0:
        variable_names.append('service_reliability')
    elif sum(variable_df.iloc[a][variable['price']]) > 0:
        variable_names.append('price')
    elif sum(variable_df.iloc[a][variable['problem_solving']]) > 0:
        variable_names.append('problem_solving')
    elif sum(variable_df.iloc[a][variable['meet_customer_needs']]) > 0:
        variable_names.append('meet_customer_needs')
    else:
        variable_names.append('not_included')

variable_names

['personal_interaction',
 'not_included',
 'policy',
 'not_included',
 'not_included',
 'physical_aspect',
 'physical_aspect',
 'not_included',
 'aesthetics',
 'not_included',
 'not_included',
 'not_included',
 'not_included',
 'not_included',
 'not_included',
 'not_included',
 'not_included',
 'problem_solving']

In [104]:
variable_df['variable'] = variable_names
variable_df = variable_df.reset_index()
dimensions = []
for a in variable_df['variable']:
    if a in dimension_dict['high_quality']:
        dim = 'high_quality'
    elif a in dimension_dict['good_value']:
        dim = 'good_value'
    elif a in dimension_dict['stands_behind']:
        dim = 'stands_behind'
    elif a in dimension_dict['meet_customer_needs']:
        dim = 'meet_customer_needs'
    else:
        dim = 'not_included'
    dimensions.append(dim)
variable_df['dimensions'] = dimensions
#variable_df[[0, 'variable']].to_csv('variable_testing_30.csv', index = False)
variable_df[variable_df['variable'] == 'meet_customer_needs']

,0,appearance,sound,taste,smell,packaging,design,shapes,styles,size,...,customer service,call center,support,needs,requirement,wants,satisfaction,complacency,variable,dimensions


In [105]:
variable_merge = pd.merge(sentiment_with_cluster_df, variable_df[[0, 'variable', 'dimensions']], left_on = 'cluster', right_on = 0, how = 'right')
varible_groupby_df = variable_merge[['dimensions','variable', 'pos_or_neg']].groupby(by = ['dimensions', 'variable']).sum('pos_or_neg')/variable_merge[['dimensions', 'variable', 'pos_or_neg']].groupby(by = ['dimensions','variable']).count()*100
varible_groupby_df = varible_groupby_df.reset_index()
varible_groupby_df = varible_groupby_df[varible_groupby_df['variable'] != 'not_included']
varible_groupby_df

,dimensions,variable,pos_or_neg
0,high_quality,aesthetics,50.0
1,high_quality,personal_interaction,62.5
2,high_quality,physical_aspect,50.0
3,high_quality,policy,50.0
5,stands_behind,problem_solving,0.0


In [ ]:
import seaborn as sns
ax = sns.barplot(x="variable", y="pos_or_neg", data=varible_groupby_df)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
ax.axhline(50)

#### Final Result

In [ ]:
final_result_df = varible_groupby_df.groupby(by = 'dimensions').mean().reset_index()
#final_result_df.to_csv(r"C:\Users\Marselo\Documents\Github2\trustpilot2\online\results\costco.csv", index = False)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 6))
splot=sns.barplot(x="dimensions",y="pos_or_neg",data= final_result_df)
for p in splot.patches:
    splot.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')
plt.axhline(50)
plt.xlabel("Dimensions", size=14)
plt.ylabel("% of positive sentiment", size=14)
plt.title("Corporate Reputation of Company X ")

In [ ]:
jcn_score('piece', 'want')

In [ ]:
jcn_score_2(wn.synset('guarantee.n.01'), 'assurance')

In [ ]:
wn.synset('monkey.n.01').definition(), wn.synset('ape.n.01').definition()

In [ ]:
wn.synset('ape.n.01').definition()

In [ ]:
for a in list(synset_generator('guarantee')):
    print(a.pos() == 'n', a)